In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler,minmax_scale
data_pd=pd.read_csv("SH600519.csv")["open"]
print(data_pd)

0         88.702
1         87.355
2         84.235
3         84.592
4         83.871
          ...   
2421    1221.000
2422    1221.020
2423    1206.000
2424    1250.000
2425    1248.000
Name: open, Length: 2426, dtype: float64


In [2]:
train_set_values=data_pd[0:2426-300].values
test_set_values=data_pd[2426-300:].values
train_set_values=np.reshape(train_set_values,(len(train_set_values),1))
test_set_values=np.reshape(test_set_values,(len(test_set_values),1))
sc=MinMaxScaler(feature_range=(0, 1))
train_set_values=sc.fit_transform(train_set_values)
test_set_values=sc.fit_transform(test_set_values)
print(len(train_set_values))
print(len(test_set_values))

2126
300


In [3]:
train_x=[]
train_y=[]
for i in range(60,len(train_set_values)):
    train_x.append(train_set_values[i-60:i,0])
    train_y.append(train_set_values[i,0])
train_x=np.reshape(train_x,(len(train_x),60,1))
train_y=np.array(train_y)
test_x=[]
test_y=[]
for i in range(60,len(test_set_values)):
    test_x.append(test_set_values[i-60:i,0])
    test_y.append(test_set_values[i,0])
test_x=np.reshape(test_x,(len(test_x),60,1))
test_y=np.array(test_y)
print(test_x.shape)
print(test_y.shape)
print(train_x.shape)
print(train_y.shape)

(240, 60, 1)
(240,)
(2066, 60, 1)
(2066,)


In [4]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(100,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.mean_squared_error
            ,metrics=["accuracy"]
             )
checkpoint_save_path="./股票预测/stock_LSTM"
if os.path.exists(checkpoint_save_path+".index"):
    print("-----load model-----")
    model.load_weights(checkpoint_save_path)
stock_callbacks=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                  save_best_only=True,
                                                 save_weights_only=True,
                                                 monitor="loss")
train_history=model.fit(train_x, train_y, batch_size=64, epochs=50, validation_data=(test_x, test_y), validation_freq=1,
                    callbacks=[stock_callbacks])
print(train_history)

Train on 2066 samples, validate on 240 samples
Epoch 1/50
2066/2066 [==============================] - 27s 13ms/sample - loss: 0.0144 - accuracy: 4.8403e-04 - val_loss: 0.0090 - val_accuracy: 0.0042
Epoch 2/50
2066/2066 [==============================] - 10s 5ms/sample - loss: 0.0017 - accuracy: 4.8403e-04 - val_loss: 0.0055 - val_accuracy: 0.0042
Epoch 3/50
2066/2066 [==============================] - 11s 6ms/sample - loss: 0.0018 - accuracy: 4.8403e-04 - val_loss: 0.0054 - val_accuracy: 0.0042
Epoch 4/50
2066/2066 [==============================] - 12s 6ms/sample - loss: 0.0014 - accuracy: 4.8403e-04 - val_loss: 0.0054 - val_accuracy: 0.0042
Epoch 5/50
2066/2066 [==============================] - 11s 5ms/sample - loss: 0.0016 - accuracy: 4.8403e-04 - val_loss: 0.0051 - val_accuracy: 0.0042
Epoch 6/50
2066/2066 [==============================] - 13s 6ms/sample - loss: 0.0014 - accuracy: 4.8403e-04 - val_loss: 0.0056 - val_accuracy: 0.0042
Epoch 7/50
2066/2066 [========================